In [1]:
import sib_ldsc_cython as ld

In [2]:
import numpy as np
import glob
import pandas as pd
from scipy.optimize import minimize
import scipy.optimize
from scipy.special import comb
from scipy.misc import derivative
import scipy.stats

In [3]:
from importlib import reload
reload(ld)

<module 'sib_ldsc_cython' from '/disk/homedirs/nber/harij/gitrepos/SNIPar/ldsc_reg/cython/sib_ldsc_cython.cpython-37m-x86_64-linux-gnu.so'>

In [4]:
V = np.array([[0.5, 0.25], [0.25, 0.5]])

In [5]:
ld.Vmat2V(V, 100)

array([50. , 50. ,  0.5])

In [6]:
ld._log_ll(np.array([0.5, 0.5, 0.5]), np.array([0.1, 0.1]), V, 0.5, 100)

-1.8378770664093453

In [7]:
ld._grad_ll_v(np.array([0.5, 0.5, 0.5]), np.array([0.1, 0.1]), V, 0.5, 100)

array([ 0.,  0., -0.])